In [1]:
from collections import defaultdict
from itertools import combinations
from tqdm import tqdm

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

lines = data.strip().split("\n")

https://adventofcode.com/2024/day/23

In [3]:
## Part 1
# Given a list of undirected edges between nodes,
#  find all sets of 3 fully-connected computers where at least one starts with 't'
neighbours = defaultdict(set)
for line in lines:
    a, b = line.split("-")
    neighbours[a].add(b)
    neighbours[b].add(a)

t_nodes = set(a for a in neighbours.keys() if a.startswith('t'))
len(t_nodes)

4

In [4]:
triples = set()
for a in tqdm(neighbours):
    if a not in t_nodes:
        continue

    for b, c in combinations(neighbours[a], 2):
        if c in neighbours[b]:
            # Add triples in a canonical order to avoid dupes
            triples.add(tuple(sorted((a, b, c))))

len(triples)

100%|██████████| 16/16 [00:00<00:00, 27799.86it/s]


7

In [5]:
## Part 2
# Find the largest fully-connected set of computers
# Apparently this is called a clique, and has an implementation in networkx
# https://stackoverflow.com/a/79250363
# https://en.wikipedia.org/wiki/Clique_problem

# def is_fully_connected(nodes):
#     return all(n2 in neighbours[n1] for n1, n2 in combinations(nodes, 2))

# Attempt 1 (quick and dirty):
# This is a relatively small graph, and we already know the number of neighbours of each node
# Find a reasonable start-node that may be fully-connected. Its neighbour length should be near the mode neighbour length
# -> Infeasible in this form. All nodes have 13 neighbours, but not all connected
# len_ns = sorted((len(ns), a) for a, ns in neighbours.items())
# len_ns

In [6]:
# Bron-Kerbosch algorithm without pivoting to find all maximal cliques (not contained in a larger clique)
# https://en.wikipedia.org/wiki/Bron%E2%80%93Kerbosch_algorithm
def bron_kerbosch(r: set, p: set, x: set):
    # R: Required, P: Possible, X: Excluded nodes
    if (not p) and (not x):
        # R is a maximal clique
        yield r
    for v in p:
        yield from bron_kerbosch(r | {v}, p & neighbours[v], x & neighbours[v])
        p = p - {v}
        x = x | {v}

maximal_cliques = list(bron_kerbosch(set(), set(neighbours.keys()), set()))
len(maximal_cliques)

15

In [7]:
# Find the largest clique
biggest_clique = max(maximal_cliques, key=len)
password = ",".join(sorted(biggest_clique))
password

'co,de,ka,ta'